In [66]:
import random
import json
from pprint import pprint
import nltk
import re
import string
from nltk.tag.stanford import StanfordPOSTagger
from nltk.tag.stanford import StanfordNERTagger
from nltk.chunk import RegexpParser
import nertclient

### import Standford POS Tagger and Stanford NER Tagger

In [67]:
#to run StanfordPostTagger and NERTagger, 
#first download these two packages from http://nlp.stanford.edu/software/CRF-NER.shtml 
#I saved the downloaded files in lib/
post = StanfordPOSTagger('lib/stanford-postagger-2014-08-27/models/english-bidirectional-distsim.tagger', 
                         'lib/stanford-postagger-2014-08-27/stanford-postagger.jar', 'utf-8') # doctest: +SKIP
# post.tag('What is the airspeed of an unladen swallow ?'.split()) # doctest: +SKIP
nert = StanfordNERTagger('lib/stanford-ner-2014-08-27/classifiers/english.all.3class.distsim.crf.ser.gz',
                         'lib/stanford-ner-2014-08-27/stanford-ner.jar', 'utf-8')

### make a random sample of 2 and smaple of 5 from the json data as starting data.
Note: only run this code once. Currently commented out. 

In [28]:
# with open('movies-with-roles-summaries.json')as data_file:
#     data = json.load(data_file)

# print(type(data))

In [29]:
# data_sample_2 = random.sample((data.items()), 2)

In [30]:
# data_sample_5 = random.sample((data.items()), 5)

In [31]:
# with open('data_sample_2.json', 'w') as outfile:
#     json.dump(data_sample_2, outfile)

In [32]:
# with open('data_sample_5.json', 'w') as outfile:
#     json.dump(data_sample_5, outfile)

###Load in the sample of five

In [68]:
with open('data_sample_5.json') as sample:
# with open('movies-with-roles-summaries.json') as sample:
    sample = json.load(sample)

###More data cleaning

In [69]:
# get rid of ([[ in the wikipidia summaries
for movie in sample:
    summaries_wiki = movie[1]['summaries_wikipedia']
    if len(summaries_wiki) > 0:
        summaries_wiki = re.sub(r'\s\(\[\[', ' ', summaries_wiki[0])
        movie[1]['summaries_wikipedia'][0] = summaries_wiki

In [70]:
sample[3]


['flirtation walk.1934',
 {'id': 34573,
  'language': None,
  'num_ratings': '388',
  'ratings': '5.8',
  'roles': [{'gender': 'F', 'role': 'Chorus Girl'},
   {'gender': 'F', 'role': 'Chorus Girl'},
   {'gender': 'F', 'role': 'Dowager at Show'},
   {'gender': 'F', 'role': 'Redhead'},
   {'gender': 'F', 'role': 'Chorus Girl'},
   {'gender': 'F', 'role': "'Kit' Fitts"},
   {'gender': 'F', 'role': 'Blonde'},
   {'gender': 'F', 'role': 'Chorus Girl'},
   {'gender': 'F', 'role': 'Girl'},
   {'gender': 'F', 'role': 'Chorus Girl'},
   {'gender': 'M', 'role': "'Oskie'"},
   {'gender': 'M', 'role': "'Spike'"},
   {'gender': 'M', 'role': 'Cadet'},
   {'gender': 'M', 'role': "'Eight Ball'"},
   {'gender': 'M', 'role': 'Native Leader'},
   {'gender': 'M', 'role': 'Gen. Landacre'},
   {'gender': 'M', 'role': 'Cadet'},
   {'gender': 'M', 'role': 'Chase'},
   {'gender': 'M', 'role': "Fitts' Butler"},
   {'gender': 'M', 'role': 'Lieut. Biddle'},
   {'gender': 'M', 'role': 'Bandleader'},
   {'gender': 

###Tagging

In [71]:
#pos tagging using nltk.pos_tag
#ner tagging using stanford ner tagger
def ie_preprocess(document, lower='false', stage="pos"):
#     if stage == 'pos':
#         sentences = nltk.sent_tokenize(document)
#         sentences = [nltk.word_tokenize(sent) for sent in sentences]
#         if lower == 'false':
#             for sent in sentences:
#                 for i in range(len(sent)):
#                     if sent[i] != sent[i].lower():
#                         sent[i] = sent[i].lower()
#         sentences = [nltk.pos_tag(sent) for sent in sentences]
#         return sentences
    if stage == "ner":
        return nert.tag(document.split())

In [ ]:
# make stanford nert more accurate by seperating the last word
# in a sentence from the ending punctuation with a space.
def insert_space_before_punct(s):
    # return re.sub(r'([-\/#!$%\^&\*;:{}=\-_`~()])', r' \1', s)
    # s = re.sub(r'([-\/#$%\^&\*{}=\-_`~()])', r'', s)
    s = re.sub(r'([.,!?;:])', r' \1', s)
    return s

In [72]:
# pos + ner tag summaries and roles
for movie in sample:
    summaries_imdb = movie[1]["summaries_imdb"]
    summaries_wiki = movie[1]['summaries_wikipedia']
    roles = movie[1]['roles']
    if len(summaries_imdb) > 0:
        # sample[i][1]["summaries_imdb_pos"] = ie_preprocess(summaries_imdb[0])
        sample[i][1]["summaries_imdb_ner"] = ie_preprocess(insert_space_before_punct(summaries_imdb[0]), stage="ner")
        
    if len(summaries_wiki) > 0:
        # sample[i][1]["summaries_wikipedia_pos"] = ie_preprocess(summaries_wiki[0])
        sample[i][1]["summaries_wikipedia_ner"] = ie_preprocess(insert_space_before_punct(summaries_wiki[0]), stage='ner')


###Search for a ner tagged name in the roles bag of words

####First, combine wiki summaries and imdb summaries into one string. 

In [73]:
# combine wiki_ner and imdb_ner
for i in range(len(sample)):
    combined = []
    if 'summaries_wikipedia_ner' in sample[i][1]:
        combined.extend( sample[i][1]['summaries_wikipedia_ner'] )
    if 'summaries_imdb_ner' in sample[i][1]:
        combined.extend(sample[i][1]["summaries_imdb_ner"])
    sample[i][1]['summaries_combined_ner'] = combined


# for movie in sample:
# #     summaries_imdb_ner is indeed a list
#     summaries_imdb_ner = movie[1]["summaries_imdb_ner"]
#     if 'summaries_wikipedia_ner' in movie[1]:
#         summaries_wiki_ner = movie[1]['summaries_wikipedia_ner']
#     else: summaries_wiki_ner = []
#     summaries_imdb_ner.extend(summaries_wiki_ner)
#     movie[1]['summaries_combined_ner'] = summaries_imdb_ner

####Run chuncker to lift out tagged names from summaries

In [74]:
# Define custom tagged entities - group NE's together 
def chunker_rules(values):
    # Define  custom grammar (modified to be a valid regex).
    grammar = r'''
        PERSON:
                {<PERSON>+}
            '''
    cp = nltk.RegexpParser(grammar) # Create an instance of your custom parser.
    return cp.parse(values)         # Parse!

def entity_chunker(tagged_docs):
    chunks = []
#     for doc in tagged_docs:
    tree = chunker_rules(tagged_docs)
    for subtree in tree.subtrees():
#             if (subtree.node == 'WIDOW'):
        leaflist = [leaf[0] for leaf in subtree.leaves()]
        chunks.append(' '.join(leaflist))
    return chunks

In [75]:
# creating a new dict called sum_and_char to store summaries and corresponding characters
sum_and_char = {}
for movie in sample:
#     making a bag of words for roles
    roles_bag = []
    roles = movie[1]['roles']
    for role in roles:
        roles_bag.append(role['role'])
    str = " ".join(roles_bag)
#     split roles on space, /, ' or "
    roles_bag = re.split(r' |/|\'|\"', str)
#     sorting entity_chuncking results into summaries and characters
    result = entity_chunker(movie[1]['summaries_combined_ner'])
    movie_name = movie[0]
    if len(result) <= 1:
        char_data = {}
    else:
        char_data = set(result[1:])
    sum_and_char[movie_name] = {"sum": result[0], \
                                "char_raw": char_data,\
                                "roles_bag": set(roles_bag),\
                                "char_info" : {}}

###Filter character names, and establish the link between filtered character names from summaries and their counter parts in the roles list

In [77]:
for movie in sample:
#     making a bag of words for roles
    for role in movie[1]['roles']:
        role_words = role['role']
        role_bag = re.split(r' |/|\'|\"', role_words)
        role['role_bag'] = (set(role_bag))

In [78]:
for movie in sample:
    for role in movie[1]['roles']:
        print(role)

{'gender': 'F', 'role_bag': {'Vanessa'}, 'role': 'Vanessa'}
{'gender': 'F', 'role_bag': {'Nelie'}, 'role': 'Nelie'}
{'gender': 'F', 'role_bag': {'Kano'}, 'role': 'Kano'}
{'gender': 'F', 'role_bag': {'Cora'}, 'role': 'Cora'}
{'gender': 'F', 'role_bag': {'Bea'}, 'role': 'Bea'}
{'gender': 'F', 'role_bag': {'Gaying'}, 'role': 'Gaying'}
{'gender': 'F', 'role_bag': {'Adelina'}, 'role': 'Adelina'}
{'gender': 'F', 'role_bag': {'Mother', 's', 'Vanessa'}, 'role': "Vanessa's Mother"}
{'gender': 'F', 'role_bag': {'Virgie'}, 'role': 'Virgie'}
{'gender': 'F', 'role_bag': {'Baby'}, 'role': 'Baby'}
{'gender': 'F', 'role_bag': {'Marichu'}, 'role': 'Marichu'}
{'gender': 'M', 'role_bag': {'Adelina', 's', 'Client'}, 'role': "Adelina's Client"}
{'gender': 'M', 'role_bag': {'Manay', 'Sharon'}, 'role': 'Manay Sharon'}
{'gender': 'M', 'role_bag': {'Recruiter'}, 'role': 'Recruiter'}
{'gender': 'M', 'role_bag': {'Mall', 'couple'}, 'role': 'Mall couple'}
{'gender': 'M', 'role_bag': {'Alex'}, 'role': 'Alex'}
{'ge

In [79]:
# filter char_raw against role list and link char_names with their names in the role list.
''' 
match words in char_raw against roles in role list. If there is zero word match, discard. If there is at least one
match, keep.

char_info in sum_and_char use names in the role list as keys, and contains role gender, char names
found in summaries, and roles found in summaries

The matching selects the highest # of matches between a role and a character name. Ties are broken by selecting the role with
the fewest words.
'''
for key, value in (sum_and_char.items()):
    for movie in sample:
#     locate correct movie
        if movie[0] == key:
#             for each filted char name
            for name in value['char_raw']:
                max_count = 0
                max_role = None
                max_gen = None
#                 tie-breaker on # of words contained in a role_bag
                max_role_bag_len = 0

#                 tied_list = []
                name_split = name.split()
#                 for each role
                for role in movie[1]['roles']:
                    count = 0
#                   for each word in char name
#                   increment count by 1 every time a word in the char name appears in a role
                    for word in name_split:
                        if word in role['role_bag']:
                            count += 1
#                   select the role with the most counts!
                    if (count > max_count) or (count == max_count and len(role['role_bag']) < max_role_bag_len):
                        max_count = count
                        max_role = role['role']
                        max_gen = role['gender']
                        max_role_bag_len = len(role['role_bag'])
                
#                 After looping through all roles in the role list, decide which is the max role
                if max_count < 1:
                    print(name, "discarded")
                else:   
                    if max_role not in value['char_info']:
                        value['char_info'][max_role] = {'gender':max_gen, 'roles_found_in_sums':[], 'names_found_in_sums':[name]}
                    else:
                        value['char_info'][max_role]['names_found_in_sums'].append(name)

Satan discarded
Ishmael Bernal discarded
Mitchell discarded
Richard Palmer Grant discarded


###Output two dict (sample, and sum_and_char) as json files

In [ ]:
def set_default(obj):
    if isinstance(obj, set):
        d = {}
        for i in obj:
            d[i] = 1
        return d
        # return list(obj)
    raise TypeError

In [ ]:
with open(filename + '.sample.json', 'w') as outfile:
    json.dump(sample, outfile, ensure_ascii=False, default=set_default)
 